<a href="https://colab.research.google.com/github/hajonsoft/Quranic_Interpretation_AI_Model/blob/main/Quranic_Interpretation_AI_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Hugging Face's transformers and datasets libraries. Load a Pre-trained Model (GPT-2)

In [ ]:
# Install the necessary libraries
!pip install transformers datasets

from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load the more advanced GPT-NeoX-20B model and tokenizer
model_name = "EleutherAI/gpt-neox-20b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Ensure the tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load your dataset from a GitHub JSON file
dataset = load_dataset('json', data_files={'train': 'https://raw.githubusercontent.com/hajonsoft/Quranic_Interpretation_AI_Model/refs/heads/main/training_data1.json'})

# Example of accessing the dataset
print(dataset['train'][0])

# Preprocessing function to add labels
def preprocess_function(examples):
    input_text = [f"Question: {q} Wrong Answer: {w} Correct Answer: {c}" for q, w, c in zip(examples['question'], examples['wrong_response'], examples['correct_response'])]

    # Tokenize the input text
    tokenized_inputs = tokenizer(input_text, truncation=True, padding=True)

    # Set input_ids as labels
    tokenized_inputs['labels'] = tokenized_inputs['input_ids'].copy()

    return tokenized_inputs

# Set the pad_token explicitly for the model configuration
model.config.pad_token_id = tokenizer.eos_token_id

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,  # Adjust based on available GPU memory
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    fp16=True,  # Enable mixed precision training to save memory
)

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
)

# Train the model
trainer.train()

# Save the trained model and tokenizer
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')

# Define a function to handle responses and avoid sensitive content
def post_process_response(response):
    forbidden_phrases = ["physically chopping the hand", "5 physical prayers"]
    if any(phrase in response for phrase in forbidden_phrases):
        return "This answer may not align with the correct Quranic interpretation."
    return response

# Function to log feedback
def collect_feedback(response_id, feedback):
    with open("feedback_log.csv", "a") as f:
        f.write(f"{response_id},{feedback}\n")

# Define the function to ask a question and get a response
def ask_question(question):
    input_text = f"Question: {question} Answer:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Generate a response using the model with attention masks
    attention_mask = input_ids.ne(tokenizer.pad_token_id).long()  # Mask to ignore padding
    output = model.generate(input_ids, attention_mask=attention_mask, max_length=100)

    # Decode the response and post-process it
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    response = post_process_response(response)

    return response

# Example interaction with the model
response = ask_question("What is the meaning of cutting the hand of a thief?")
print(response)


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/457k [00:00<?, ?B/s]

...

model-00043-of-00046.safetensors:   0%|          | 0.00/910M [00:00<?, ?B/s]

model-00044-of-00046.safetensors:   0%|          | 0.00/910M [00:00<?, ?B/s]

model-00045-of-00046.safetensors:   0%|          | 0.00/604M [00:00<?, ?B/s]

model-00046-of-00046.safetensors:   0%|          | 0.00/620M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/46 [00:00<?, ?it/s]

Deploy to Gradio

In [ ]:
# Install necessary libraries (if not already installed in the first cell)
!pip install gradio huggingface_hub
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer from the saved results
tokenizer = AutoTokenizer.from_pretrained('./results')
model = AutoModelForCausalLM.from_pretrained('./results')

# Define the function to generate a response from the model
def generate_response(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = model.generate(input_ids, max_length=100, num_return_sequences=1)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Create Gradio interface
iface = gr.Interface(
    fn=generate_response,
    inputs="text",
    outputs="text",
    title="Quranic Interpretation AI Model",
    description="Type your question and interact with the model."
)

# Launch the interface
iface.launch(share=True)
